In [1]:
import os
import sys
module_path = os.path.join("../../KeyPoint-Analysis/KPG")
sys.path.insert(0, module_path)

In [2]:
from rouge_setbase import preprocess_dataset, compute_rouge, compute_rouge_max
from softF1 import softevaluation

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2024-03-21 19:27:47.671660: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 19:27:47.696620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 19:27:47.696642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 19:27:47.697342: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Una

In [3]:
import pandas as pd
import ast

# Import Data

## Prediction Data

In [4]:
prediction_df = pd.read_csv("../../data/space/kpg_summaries.csv")
prediction_df = prediction_df[~prediction_df['generated_kp'].str.contains('comments')]
prediction_df = prediction_df.iloc[:,:7].drop(columns=['aspects_terms'])
prediction_df = prediction_df.rename(columns={'cluster_sentiment': 'sentiment', 
                                              'business_id': 'entity_id',  
                                              'business_name': 'entity_name'})

In [5]:
prediction_df

,topic,entity_id,entity_name,sentiment,generated_kp,coverage
0,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,negative,Not meeting Hilton standards.,9
1,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,negative,Expensive and inconvenient parking.,8
2,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,negative,Non-functional showers.,8
3,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,negative,Poor room service and maintenance.,8
4,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,negative,Slow internet service.,7
...,...,...,...,...,...,...
369,Hotels,156564,Wedgewood Hotel & Spa,positive,Excellent choice.,2
370,Hotels,156564,Wedgewood Hotel & Spa,positive,Free in-room WiFi.,2
371,Hotels,156564,Wedgewood Hotel & Spa,positive,Loved the facial at the Spa.,2
372,Hotels,156564,Wedgewood Hotel & Spa,positive,Returning to a gracious home.,2


In [6]:
# prediction_df['num_of_token'] = prediction_df['generated_kp'].apply(lambda x: len([token for token in nlp(x) if token.pos_ not in ['PUNCT', 'SPACE']]))

In [7]:
prediction_df.shape

(358, 6)

In [8]:
# prediction_df = prediction_df[(prediction_df['num_of_token'] >= 3) & (prediction_df['num_of_token'] <= 5)]
# prediction_df = prediction_df[(prediction_df['num_of_token'] >= 3) & (prediction_df['num_of_token'] <= 6)]

In [9]:
prediction_df.shape

(358, 6)

In [10]:
# min_coverage = 1
# min_coverage = 5
# min_coverage = 10
min_coverage = 15
prediction_df = prediction_df[prediction_df['coverage'] > min_coverage]
prediction_df

,topic,entity_id,entity_name,sentiment,generated_kp,coverage
12,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Clean and spacious rooms.,73
13,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Great hotel with excellent service.,72
14,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Great location with easy access to attractions and sights.,50
15,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Friendly and helpful staff.,48
16,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Comfortable beds and good sleep quality.,30
...,...,...,...,...,...,...
339,Hotels,156564,Wedgewood Hotel & Spa,positive,Enjoyable and memorable stay.,20
341,Hotels,156564,Wedgewood Hotel & Spa,positive,Delicious cookies provided during turndown service.,17
342,Hotels,156564,Wedgewood Hotel & Spa,positive,Excellent food quality.,17
343,Hotels,156564,Wedgewood Hotel & Spa,positive,Comfortable and luxurious beds.,16


## Merge

In [11]:
summary_df = pd.read_pickle("../../data/space/space_summ_sentiment_analyzed.pkl")

In [12]:
merged_df = prediction_df.merge(summary_df, on=['entity_id', 'entity_name', 'sentiment'], how='left')
merged_df

topic  entity_id                              entity_name sentiment  \
0     Hotels    1064610  Hilton Garden Inn Montreal Centre-ville  positive   
1     Hotels    1064610  Hilton Garden Inn Montreal Centre-ville  positive   
2     Hotels    1064610  Hilton Garden Inn Montreal Centre-ville  positive   
3     Hotels    1064610  Hilton Garden Inn Montreal Centre-ville  positive   
4     Hotels    1064610  Hilton Garden Inn Montreal Centre-ville  positive   
...      ...        ...                                      ...       ...   
3397  Hotels     156564                    Wedgewood Hotel & Spa  positive   
3398  Hotels     156564                    Wedgewood Hotel & Spa  positive   
3399  Hotels     156564                    Wedgewood Hotel & Spa  positive   
3400  Hotels     156564                    Wedgewood Hotel & Spa  positive   
3401  Hotels     156564                    Wedgewood Hotel & Spa  positive   

                         generated_kp  coverage  \
0           Clean and spacious rooms.        73   
1           Clean and spacious rooms.        73   
2           Clean and spacious rooms.        73   
3           Clean and spacious rooms.        73   
4           Clean and spacious rooms.        73   
...                               ...       ...   
3397  The bar has a great atmosphere.        16   
3398  The bar has a great atmosphere.        16   
3399  The bar has a great atmosphere.        16   
3400  The bar has a great atmosphere.        16   
3401  The bar has a great atmosphere.        16   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

# Evaluation

In [13]:
pd.set_option('max_colwidth', 500)

In [14]:
merged_df = merged_df.dropna()

In [15]:
from softF1 import *

In [16]:
merged_df = merged_df[['topic', 'entity_id', 'entity_name', 'sentiment', 'generated_kp', 'gold_summary_sentences']]

In [17]:
merged_df = merged_df.rename(columns={
    'generated_kp': 'key_point',
    'topic': 'category',
    'entity_name': 'topic',
    'sentiment': 'stance',
    'gold_summary_sentences': 'key_point_given'
})

In [18]:
merged_df

,category,entity_id,topic,stance,key_point,key_point_given
0,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Clean and spacious rooms.,"The location is very central, and you can walk to anywhere, including the Place de Arts, Old Montreal, Le Main, etc."
1,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Clean and spacious rooms.,The top floor gym and swimming pool are definitely worth checking out.
2,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Clean and spacious rooms.,The staff is friendly and polite and were very helpful to recommend some great places to visit in the city.
3,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Clean and spacious rooms.,"The rooms, featuring amenities your average guest will find, and not to mention the whole hotel, is very clean."
4,Hotels,1064610,Hilton Garden Inn Montreal Centre-ville,positive,Clean and spacious rooms.,"There were some cleanliness issues with the bath and shower, though."
...,...,...,...,...,...,...
3397,Hotels,156564,Wedgewood Hotel & Spa,positive,The bar has a great atmosphere.,"Food from Bacchus was fantastic, with a good bar to accompany it."
3398,Hotels,156564,Wedgewood Hotel & Spa,positive,The bar has a great atmosphere.,It's nice to get the chocolates and cookies with turn-down.
3399,Hotels,156564,Wedgewood Hotel & Spa,positive,The bar has a great atmosphere.,The continental breakfast was average.
3400,Hotels,156564,Wedgewood Hotel & Spa,positive,The bar has a great atmosphere.,"The restaurant is quite formal, but the food is excellent."


## BARTScore

### SoftPrecision

In [19]:
softp_data = merged_df.groupby(['category', 'entity_id', 'topic', 'stance', 'key_point'])\
    .apply(lambda grp: "".join([cand + "=" for cand in (grp['key_point_given'].tolist())])).reset_index(name='multi_cands')
softp_data

,category,entity_id,topic,stance,key_point,multi_cands
0,Hotels,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"The hotel staff created a lovely, professional, and overwhelmingly helpful atmosphere.=Every room was tastefully clean and spotless.=The rooms (beds and showers included) were very comfortable and filled with first class amenities.=The continental breakfast was average, but the restaurant itself is formal, with a terrific presentation and excellent food.=The hotel is close to a movie theater, shops, and restaurants on Robson St.=The staff were also incredibly friendly, efficient, and helpful..."
1,Hotels,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious suites.,"The hotel staff created a lovely, professional, and overwhelmingly helpful atmosphere.=Every room was tastefully clean and spotless.=The rooms (beds and showers included) were very comfortable and filled with first class amenities.=The continental breakfast was average, but the restaurant itself is formal, with a terrific presentation and excellent food.=The hotel is close to a movie theater, shops, and restaurants on Robson St.=The staff were also incredibly friendly, efficient, and helpful..."
2,Hotels,156564,Wedgewood Hotel & Spa,positive,Comfortable and luxurious beds.,"The hotel staff created a lovely, professional, and overwhelmingly helpful atmosphere.=Every room was tastefully clean and spotless.=The rooms (beds and showers included) were very comfortable and filled with first class amenities.=The continental breakfast was average, but the restaurant itself is formal, with a terrific presentation and excellent food.=The hotel is close to a movie theater, shops, and restaurants on Robson St.=The staff were also incredibly friendly, efficient, and helpful..."
3,Hotels,156564,Wedgewood Hotel & Spa,positive,Delicious cookies provided during turndown service.,"The hotel staff created a lovely, professional, and overwhelmingly helpful atmosphere.=Every room was tastefully clean and spotless.=The rooms (beds and showers included) were very comfortable and filled with first class amenities.=The continental breakfast was average, but the restaurant itself is formal, with a terrific presentation and excellent food.=The hotel is close to a movie theater, shops, and restaurants on Robson St.=The staff were also incredibly friendly, efficient, and helpful..."
4,Hotels,156564,Wedgewood Hotel & Spa,positive,Delicious meals and impressive breakfast.,"The hotel staff created a lovely, professional, and overwhelmingly helpful atmosphere.=Every room was tastefully clean and spotless.=The rooms (beds and showers included) were very comfortable and filled with first class amenities.=The continental breakfast was average, but the restaurant itself is formal, with a terrific presentation and excellent food.=The hotel is close to a movie theater, shops, and restaurants on Robson St.=The staff were also incredibly friendly, efficient, and helpful..."
...,...,...,...,...,...,...
61,Hotels,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Excellent location with great service.,"Nice place to stay with great service.=It was quiet and clean.=The food was really good in a great location.=The hotel, which is close to the beach, is very classy, with a good, small restaurant and rooftop bar with a great view.=The easy-going staff is friendly the minute guests walk in, imparting excellent, helpful service with whatever need or piece of information one requests.=The rooms are very clean and homey, with comfortable beds and a view of the beach.=The food at Hash, the restaur..."
62,Hotels,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,Friendly and helpful staff.,"Nice place to stay with great service.=It was quiet and clean.=The food was really good in a great location.=The hotel, which is close to the beach, is very classy, with a good, small restaurant and rooftop bar with a great view.=The easy-going staff is friendly the minute guests wal

In [20]:
cands, refs= preprocess_text(softp_data, metrics = "softPrecision")

#BART score cannot be processed for different numbers of reference sentences, so
#check for the maximum number of reference sentences and match the size.
#We fill in the None for the missing sentences because we only pick one of the 
#maximum values and not the average, so there is no impact on performance

refs = balance_ref_num(refs)        

# generation scores from the first list of texts to the second list of texts.
P = bart_scorer.multi_ref_score(cands, refs, agg="max", batch_size=4) # agg means aggregation, can be mean or max

#mapping the score to (0,1]
P_average = math.tanh(math.exp((mean(P))/2+1.3))
#P_average = math.tanh(mean(P)) + 1
#P_average = math.exp(mean(P))

In [20]:
P_average

0.6905645553726367

In [23]:
P_average

0.7443144278583115

### SoftRecall

In [23]:
softr_data = merged_df.groupby(['category', 'entity_id', 'topic', 'stance', 'key_point_given'])\
    .apply(lambda grp: "".join([cand + "=" for cand in (grp['key_point'].tolist())])).reset_index(name='multi_cands')
softr_data

,category,entity_id,topic,stance,key_point_given,multi_cands
0,Hotel,156564,Wedgewood Hotel & Spa,positive,All the staff were extremely helpful and polite.,A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
1,Hotel,156564,Wedgewood Hotel & Spa,positive,"All the staff were extremely helpful, polite, and always had a smile.",A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
2,Hotel,156564,Wedgewood Hotel & Spa,positive,"Amenities included flat screen television sets, DVD players.",A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
3,Hotel,156564,Wedgewood Hotel & Spa,positive,"Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortable stay.",A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
4,Hotel,156564,Wedgewood Hotel & Spa,positive,Another nice touch was the chocolates and cookies with turndown.,A wonderful and quiet place.=Beautiful and spacious bathroom.=Beautiful and spacious suites.=Comfortable amenities and relaxing atmosphere.=Comfortable and luxurious beds.=Convenient location for shopping.=Delicious meals and impressive breakfast.=Elegant and well-maintained property.=Enjoyable and memorable stay.=Enjoyed the spa services.=Excellent experience at the Wedgewood.=Excellent food quality.=Free in-room WiFi.=Friendly and professional staff.=Helpful and friendly valet service.=Hel...
...,...,...,...,...,...,...
350,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,There was little privacy in the bathroom.,A home away from home.=California is a must-do.=Clean and comfortable rooms.=Close proximity to the beach.=Comfortable and luxurious towels.=Comfortable beds and pillows.=Convenient parking options available.=Delicious breakfast and great cocktails.=Delicious food at Hash restaurant.=Enjoyable and pleasant stay.=Eugene provided exceptional service.=Excellent location with great service.=Friendly and helpful staff.=Friendly bartender with great recommendations.=Funky and modern decor.=Good va...
351,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,"There was oatmeal, great wheat toast, freshberries and a tasty corned beef hash and various other items",A hom

In [24]:
cands, refs= preprocess_text(softr_data, metrics = "softRecall")

refs = balance_ref_num(refs)

R = bart_scorer.multi_ref_score(cands, refs, agg="max", batch_size=4)

#mapping the score to (0,1]
R_average = math.tanh(math.exp((mean(R)/2)+1.3)) 
#R_average = (0.25 * mean(R)) + 1
#R_average = math.exp(mean(R))

In [15]:
R_average

0.7077926347982703

In [27]:
R_average

0.6626422886324295

### F1

In [26]:
result = softF1(P_average, R_average)

In [17]:
result

0.6979538059491077

In [29]:
result

0.7011078736925943

In [17]:
softp_data

,ID,key_point,key_point_given,topic,stance,multi_cands
0,0,should not be mandatory because there are many side effects. Routine child vaccinations should not be mandatory because there are many side effects,"Routine child vaccinations, or their side effects, are dangerous",Routine child vaccinations should be mandatory.,Negative,"Routine child vaccinations, or their side effects, are dangerous=Mandatory vaccination contradicts basic rights=The parents and not the state should decide=Routine child vaccinations are not necessary to keep children healthy="
1,1,should not be mandatory,Mandatory vaccination contradicts basic rights,Routine child vaccinations should be mandatory.,Negative,"Routine child vaccinations, or their side effects, are dangerous=Mandatory vaccination contradicts basic rights=The parents and not the state should decide=Routine child vaccinations are not necessary to keep children healthy="
2,2,is a violation of human rights,The parents and not the state should decide,Routine child vaccinations should be mandatory.,Negative,"Routine child vaccinations, or their side effects, are dangerous=Mandatory vaccination contradicts basic rights=The parents and not the state should decide=Routine child vaccinations are not necessary to keep children healthy="
3,3,vaccination is not urgent because children don't suffer from symptoms,Routine child vaccinations are not necessary to keep children healthy,Routine child vaccinations should be mandatory.,Negative,"Routine child vaccinations, or their side effects, are dangerous=Mandatory vaccination contradicts basic rights=The parents and not the state should decide=Routine child vaccinations are not necessary to keep children healthy="
4,4,is one of the most effective public health interventions for saving lives and promoting good health,Routine child vaccinations are effective,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer from preventable diseases=
5,5,saves lives,Child vaccination saves lives,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer from preventable diseases=
6,6,vaccinations are necessary to ensure that the population are protected from communicable diseases,Routine child vaccinations are necessary to protect others,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer from preventable diseases=
7,7,should be mandatory to avoid spreading the virus. Routine child vaccinations should be mandatory to avoid spreading the virus,Routine child vaccinations should be mandatory to prevent virus/disease spreading,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer from preventable diseases=
8,8,prevents diseases in children. Vaccines prevent diseases from childhood,Children should not suffer from preventable diseases,Routine child vaccinations should be mandatory.,Positive,Routine child vaccinations are effective=Child vaccination saves lives=Routine child vaccinations are necessary to protect others=Routine child vaccinations should be mandatory to prevent virus/disease spreading=Children should not suffer fro

## BLEURTScore

In [25]:
softp_data = merged_df.sort_values(by=['category', 'entity_id', 'topic', 'stance', 'key_point'])
df_compare_precision = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point': 'candidate', 'key_point_given': 'reference'})
df_compare_precision

,candidate,reference
1,Beautiful and spacious bathroom.,"The hotel staff created a lovely, professional, and overwhelmingly helpful atmosphere."
2,Beautiful and spacious bathroom.,Every room was tastefully clean and spotless.
3,Beautiful and spacious bathroom.,The rooms (beds and showers included) were very comfortable and filled with first class amenities.
4,Beautiful and spacious bathroom.,"The continental breakfast was average, but the restaurant itself is formal, with a terrific presentation and excellent food."
5,Beautiful and spacious bathroom.,"The hotel is close to a movie theater, shops, and restaurants on Robson St."
...,...,...
3398,The rooftop bar has amazing views.,"The inhouse restaurant was healthy and superb, and geared towards customer comfort."
3399,The rooftop bar has amazing views.,"Breakfast included oatmeal, great wheat toast, fresh berries and a tasty corned beef hash."
3400,The rooftop bar has amazing views.,"Room service for breakfast was extremely fast, accurate and tasty."
3401,The rooftop bar has amazing views.,"The food at Hash, the restaurant just off of the lobby, was fabulous for breakfast."


In [26]:
candidates = df_compare_precision['candidate']
references = df_compare_precision['reference']

In [28]:
import tensorflow as tf

In [29]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [31]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9709736529553824590
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14626652160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1620072431990812415
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [32]:
with tf.device('/gpu:0'):
  result = calculatingScore(references, candidates)
  df_compare_precision["BLEURT Score"] = result

  #After calculating the semantic quality of all candidates and reference pairs, the one with the highest score is selected as the correct pair.
  df_bestkp_pair_precision = df_compare_precision.loc[df_compare_precision.groupby(["candidate"])["BLEURT Score"].idxmax()]
  #take average of all best scores as the soft precision score.
  P_average = df_bestkp_pair_precision["BLEURT Score"].mean()

### SoftPrecision

In [21]:
softp_data = merged_df.sort_values(by=['category', 'entity_id', 'topic', 'stance', 'key_point'])
df_compare_precision = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point': 'candidate', 'key_point_given': 'reference'})
df_compare_precision

,candidate,reference
1,Beautiful and spacious bathroom.,"The hotel staff created a lovely, professional, and overwhelmingly helpful atmosphere."
2,Beautiful and spacious bathroom.,Every room was tastefully clean and spotless.
3,Beautiful and spacious bathroom.,The rooms (beds and showers included) were very comfortable and filled with first class amenities.
4,Beautiful and spacious bathroom.,"The continental breakfast was average, but the restaurant itself is formal, with a terrific presentation and excellent food."
5,Beautiful and spacious bathroom.,"The hotel is close to a movie theater, shops, and restaurants on Robson St."
...,...,...
3398,The rooftop bar has amazing views.,"The inhouse restaurant was healthy and superb, and geared towards customer comfort."
3399,The rooftop bar has amazing views.,"Breakfast included oatmeal, great wheat toast, fresh berries and a tasty corned beef hash."
3400,The rooftop bar has amazing views.,"Room service for breakfast was extremely fast, accurate and tasty."
3401,The rooftop bar has amazing views.,"The food at Hash, the restaurant just off of the lobby, was fabulous for breakfast."


In [22]:
candidates = df_compare_precision['candidate']
references = df_compare_precision['reference']

In [23]:
import tensorflow as tf

In [24]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-02-03 18:13:14.783059: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:14.805999: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:14.806189: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [25]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18435889184579564257
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22356492288
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5549650304347576173
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:07:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-02-03 18:13:16.587623: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:16.587776: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:16.587907: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [26]:
result = calculatingScore(references, candidates)
df_compare_precision["BLEURT Score"] = result

#After calculating the semantic quality of all candidates and reference pairs, the one with the highest score is selected as the correct pair.
df_bestkp_pair_precision = df_compare_precision.loc[df_compare_precision.groupby(["candidate"])["BLEURT Score"].idxmax()]
#take average of all best scores as the soft precision score.
P_average = df_bestkp_pair_precision["BLEURT Score"].mean()

INFO:tensorflow:Reading checkpoint BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: BLEURT-20/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


2024-02-03 18:13:24.502151: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:24.502354: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:13:24.502494: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [27]:
P_average

0.6124507463445851

### SoftRecall

In [33]:
softr_data = merged_df.sort_values(by=['category', 'entity_id', 'topic', 'stance', 'key_point_given'])
df_compare_recall = softp_data[['key_point', 'key_point_given']].rename(columns={'key_point_given': 'candidate', 'key_point': 'reference'})
df_compare_recall

,reference,candidate
36,Beautiful and spacious bathroom.,All the staff were extremely helpful and polite.
95,Beautiful and spacious suites.,All the staff were extremely helpful and polite.
154,Comfortable and luxurious beds.,All the staff were extremely helpful and polite.
213,Delicious cookies provided during turndown service.,All the staff were extremely helpful and polite.
272,Delicious meals and impressive breakfast.,All the staff were extremely helpful and polite.
...,...,...
3133,Excellent location with great service.,Very close to Venice Beach and a long-ish walk to Santa Monica.
3193,Friendly and helpful staff.,Very close to Venice Beach and a long-ish walk to Santa Monica.
3253,Highly recommended hotel with great experience.,Very close to Venice Beach and a long-ish walk to Santa Monica.
3313,Stunning ocean views.,Very close to Venice Beach and a long-ish walk to Santa Monica.


In [34]:
candidates = df_compare_recall['candidate']
references = df_compare_recall['reference']

In [35]:
import tensorflow as tf

In [36]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [37]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3616641842038152970
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22356492288
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2208089500298092512
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:07:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-02-03 18:21:53.444547: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:21:53.444746: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-03 18:21:53.444884: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [38]:
result = calculatingScore(references, candidates)
df_compare_recall["BLEURT Score"] = result

#After calculating the semantic quality of all candidates and reference pairs, the one with the highest score is selected as the correct pair. 
df_bestkp_pair_recall = df_compare_recall.loc[df_compare_recall.groupby(["candidate"])["BLEURT Score"].idxmax()]

#take average of all best scores as the soft precision score.
R_average = df_bestkp_pair_recall["BLEURT Score"].mean()

INFO:tensorflow:Reading checkpoint BLEURT-20.


INFO:tensorflow:Reading checkpoint BLEURT-20.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint BLEURT-20


...

INFO:tensorflow:Creating Eager Mode predictor.


INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [39]:
R_average

0.5088073847170212

### F1

In [40]:
result = softF1(P_average, R_average)

In [41]:
result

0.555838934644844

## ROUGE

In [51]:
gt_gold_kp = merged_df

In [52]:
gt_gold_kp

,category,entity_id,topic,stance,key_point,key_point_given,P,R
1,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"The rooms are beautiful, spacious, and quite.",0.475561,0.475597
2,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"Comfortable amenities abound, including full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms with great showers.",0.367027,0.367069
3,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"The beds are amazingly comfortable, with top end linens.",0.305698,0.305746
4,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,These quiet and spacious and luxurious rooms had wonderful European feel with great showers.,0.523816,0.523848
5,Hotel,156564,Wedgewood Hotel & Spa,positive,Beautiful and spacious bathroom.,"Amenities such as a full living room furniture, DVD players for the flat-screen televisions, and large marble bathrooms made for a comfortable stay.",0.342050,0.342096
...,...,...,...,...,...,...,...,...
2331,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"The inhouse restaurant was healthy and superb, and geared towards customer comfort.",0.447668,0.447706
2332,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"Breakfast included oatmeal, great wheat toast, fresh berries and a tasty corned beef hash.",0.227394,0.227448
2333,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"Room service for breakfast was extremely fast, accurate and tasty.",0.342214,0.342261
2334,Hotel,2516007,"Hotel Erwin, a Joie de Vivre hotel",positive,The rooftop bar has amazing views.,"The food at Hash, the restaurant just off of the lobby, was fabulous for breakfast.",0.398586,0.398628


In [53]:
predictions, references = [], []
for topic in sorted(gt_gold_kp['topic'].unique()):
    for stance in ['negative', 'positive']:
        kps = gt_gold_kp.loc[(gt_gold_kp['topic']==topic) & (gt_gold_kp['stance']==stance), 'key_point'].unique().tolist()
        gold_kps = gt_gold_kp.loc[(gt_gold_kp['topic']==topic) & (gt_gold_kp['stance']==stance), 'key_point_given'].unique().tolist()
        if len(kps) > 0 and len(gold_kps) > 0:
            predictions.append(kps)
            references.append(gold_kps)

In [40]:
compute_rouge_max(predictions, references)

Rouge 1: 0.648
Rouge 2: 0.364
Rouge L: 0.51
